### Initialization

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# turn off tensorflow deprecation warnings
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

### Some helper functions

In [ ]:
def evaluate_model(model, features, labels):
    _, accuracy = model.evaluate(features, labels, verbose=0)
    print('Accuracy: %.2f' % (accuracy*100))

    # make class predictions with the model
    predictions = model.predict_classes(features)
    print("Confusion Matrix :")   
    print(confusion_matrix(labels, predictions))
    
def plot_history(history):
    # to handle metrics keys changes in Keras 2.3
    # see https://github.com/keras-team/keras/releases/tag/2.3.0
    pre_23 = "acc" in history.history.keys()
    acc_key     = 'acc'     if pre_23 else 'accuracy'
    val_acc_key = 'val_acc' if pre_23 else 'val_accuracy'       
    
    acc = history.history[acc_key]
    val_acc = history.history[val_acc_key]
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # "bo" is for "blue dot"
    plt.plot(epochs, loss, 'bo', label='Training loss')
    # b is for "solid blue line"
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

### Build a model with a variable number of layers

In [ ]:
def make_model(num_inputs, layers):
    model = Sequential()

    # first layer
    model.add(Dense(layers[0], input_dim=num_inputs, activation='relu'))

    #  intermediate layers
    for i in range(1, len(layers)):
        model.add(Dense(layers[i], activation='relu'))
    
    # final layer with a single node
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    
    return model

### Data Input
Read the bank data set and split into a features and a label subset

In [ ]:
data_sets = ('bank-10percent', 'bank-full', 'bank-balanced')

bank = pd.read_csv('../data/' + data_sets[1] + '.csv')

label_col = 'y'
label = bank[label_col]
features = bank.drop(columns=['y'])

label_encoded = pd.get_dummies(label, drop_first = True)
features_encoded = pd.get_dummies(features, drop_first = True)
feature_count=features_encoded.shape[1]

### Preprocess Data
It's recommended to normalize the data

In [ ]:
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features_encoded)

Split into training and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_normalized, label_encoded, test_size = 0.2, random_state = 167)

### Keras Model

In [ ]:
model = make_model(feature_count, [16, 8])

### Training and Evaluation

In [ ]:
num_epochs=10
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=num_epochs, 
                    verbose=1)

Plot the loss function

In [ ]:
plot_history(history)

### Evaluating model on training data

In [ ]:
evaluate_model(model, X_train, y_train)

### Evaluating model on test data

In [ ]:
evaluate_model(model, X_test, y_test)